In [1]:
import calendar
import pandas as pd
import panel as pn
from datetime import date, timedelta
from sqlalchemy import create_engine
from itables import show
pn.extension('tabulator')

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

engine = create_engine("sqlite:///c:\\ruby\\portmy\\db\\development.sqlite3")
conmy = engine.connect()

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"

today = date.today()
today

datetime.date(2022, 3, 26)

### Tables in the process

In [2]:
cols = 'trade name qty price active reason market xdate'.split()
colt = 'trans name spd reason qty target current percent active xdate'.split()

In [3]:
format_dict = {
    'qty':'{:,}','available_qty':'{:,}',
    'price':'{:.2f}','cost':'{:.2f}','max_price':'{:.2f}','min_price':'{:.2f}',
    'buy_target':'{:.2f}','sell_target':'{:.2f}','target':'{:.2f}','current':'{:.2f}',
    'beta':'{:.2f}',
    'amount':'{:,.2f}','sell_amt':'{:,.2f}','volume':'{:,.2f}',
    'percent':'{:,.2f}',
    'shares':'{:,}',    
    'q4':'{:.4f}','q3':'{:.4f}','q2':'{:.4f}','q1':'{:.4f}','dividend':'{:.4f}',
    'xdate':'{:%Y-%m-%d}','paiddate':'{:%Y-%m-%d}',
}

In [4]:
sql = '''
SELECT *
FROM orders
ORDER BY id DESC
LIMIT 1'''
orders = pd.read_sql(sql, conlite)
orders

,id,trade,name,qty,price,active,reason,market,xdate
0,26,S,WHART,20000,12.4,0,DOS,SET,2022-03-09


In [5]:
names = orders['name']
name = names.to_string(index=False)

In [6]:
sql = '''
SELECT *
FROM dividend
WHERE name = '%s'
'''
sql = sql % name
dividend = pd.read_sql(sql, const)
dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
dividend.columns = dividend.columns.str.lower()
dividend['shares'] = dividend['shares'].astype('int64')
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend['paiddate'] = pd.to_datetime(dividend['paiddate'])
dividend.style.format(format_dict)

,name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
0,WHART,0.1195,0.2553,0.1915,0.1915,0.7578,"20,000",2022-03-09,2022-03-25,,0


In [7]:
sql = """
SELECT * FROM stocks WHERE name = '%s'
"""
sql = sql % name
print(sql)

stocks = pd.read_sql(sql, conlite)
stocks.style.format(format_dict)


SELECT * FROM stocks WHERE name = 'WHART'



,id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
0,687,WHART,14.08,11.10,I,0.00,12.40,24.28,0.18,12.30,"20,000",-5,4,"20,000",0,0,DOS,SET


### Create orders from stocks after adjust stocks in port_lite from consensus

In [8]:
sql = '''
DELETE FROM orders'''
rp = conlite.execute(sql)
rp.rowcount

27

In [9]:
sql = '''
SELECT status AS trade, name, qty, buy_target AS price, 0 AS active, reason, market
FROM stocks
WHERE status IN ('B','O')
ORDER BY name'''
df_buy = pd.read_sql(sql, conlite)
df_buy

,trade,name,qty,price,active,reason,market
0,O,AMATA,9000,20.60,0,3L,SET100
1,O,ASIAN,10000,15.00,0,6L,SET
2,O,ASK,3000,42.00,0,1L,SET
3,O,BAM,12000,19.80,0,3L,SET100
4,O,BBL,2400,130.00,0,1L,SET50
5,O,BDMS,12000,23.20,0,1L,SET50
6,B,DOHOME,3000,19.70,0,RD15pct,SET100
7,O,GLOBAL,9000,19.40,0,,SET50
8,O,GULF,6000,50.00,0,1L,SET50
9,O,KKP,3000,66.00,0,3L,SET100


In [10]:
sql = '''
SELECT status AS trade, name, qty, sell_target AS price, 0 AS active, reason, market
FROM stocks
WHERE status IN ('I','S')'''
df_sell = pd.read_sql(sql, conlite)
df_sell

,trade,name,qty,price,active,reason,market
0,I,JASIF,10000,11.30,0,DOS,SET
1,I,HREIT,40100,9.00,0,DOS,SET
2,I,RATCH,6000,46.00,0,DOS,SET50
3,I,BCH,5000,22.40,0,5pct,SET100
4,I,PTT,3000,41.25,0,5pct,SET50
5,I,SINGER,1500,52.75,0,25pct,SET100
6,I,KCE,1000,68.00,0,CUT,SET50
7,I,RCL,4000,47.50,0,CP1S,SET100
8,I,WHART,20000,12.40,0,DOS,SET
9,I,CPNCG,20000,12.50,0,DOS,SET


In [11]:
df_order = df_buy.append(df_sell, ignore_index=True)
df_order.shape

(37, 7)

In [12]:
df_order.groupby(['trade']).count()

,name,qty,price,active,reason,market
trade,,,,,,
B,6,6,6,6,6,6
I,12,12,12,12,12,12
O,19,19,19,19,19,19


In [13]:
df_order.loc[df_order.trade == 'I','trade'] = 'S'
df_order.groupby(['trade']).count()

,name,qty,price,active,reason,market
trade,,,,,,
B,6,6,6,6,6,6
O,19,19,19,19,19,19
S,12,12,12,12,12,12


In [14]:
sql = '''
SELECT name, xdate
FROM dividend
'''
dividend = pd.read_sql(sql, const)
dividend.columns = dividend.columns.str.lower()
dividend['xdate'] = pd.to_datetime(dividend['xdate'])
dividend.shape

(81, 2)

In [15]:
df_merge = pd.merge(df_order, dividend, on='name', how='outer', indicator=True)
df_merge.shape

(81, 9)

### If there is no dividend record, must create dividend record before proceed

In [16]:
no_div = df_merge[df_merge['_merge'] == 'left_only']
no_div

,trade,name,qty,price,active,reason,market,xdate,_merge


In [17]:
mask = df_merge['_merge'] == 'both'
df_orders = df_merge[mask].copy()
df_orders.drop(['_merge'], axis=1, inplace=True)
df_orders['qty'] = df_orders['qty'].astype('int64')
df_orders['active'] = df_orders['active'].astype('int64')
df_orders['xdate'] = df_orders['xdate'].astype('str')
df_orders.groupby('trade').count()

,name,qty,price,active,reason,market,xdate
trade,,,,,,,
B,6,6,6,6,6,6,6
O,19,19,19,19,19,19,19
S,12,12,12,12,12,12,12


### Set xdate 2 weeks window of first Tuesday and second Friday

In [18]:
fm_date = '2022-03-29'
to_date = '2022-04-15'

### Restart and Run All Cells

In [19]:
fltr_sts_o = (df_orders.trade == 'O') & (df_orders.xdate >= fm_date) 
df_sts_o = df_orders[fltr_sts_o]
df_sts_o

,trade,name,qty,price,active,reason,market,xdate
0,O,AMATA,9000,20.6,0,3L,SET100,2022-04-25
1,O,ASIAN,10000,15.0,0,6L,SET,2022-04-29
3,O,BAM,12000,19.8,0,3L,SET100,2022-04-29
4,O,BBL,2400,130.0,0,1L,SET50,2022-04-22
9,O,KKP,3000,66.0,0,3L,SET100,2022-04-28
12,O,ORI,18000,10.8,0,3L,SET100,2022-05-09
16,O,SPALI,3000,20.4,0,RD05pct,SET100,2022-04-27
19,O,SVI,15000,8.0,0,6L,SET,2022-05-05


In [20]:
fltr_sts_b = (df_orders.trade == 'B') 
df_sts_b = df_orders[fltr_sts_b]
df_sts_b

,trade,name,qty,price,active,reason,market,xdate
6,B,DOHOME,3000,19.70,0,RD15pct,SET100,2022-02-02
10,B,MAKRO,1500,39.00,0,RD05pct,SET,2022-03-03
17,B,STA,2500,26.25,0,RD20pct,SET100,2022-04-19
20,B,SYNEX,3000,23.80,0,RD15pct,SET100,2022-03-10
21,B,TISCO,500,97.75,0,ROUND,SET50,2022-04-29
22,B,TMT,15000,11.00,0,ROUND,SET,2022-04-18


In [21]:
fltr_sts_s = (df_orders.trade == 'S')
df_sts_s = df_orders[fltr_sts_s]
df_sts_s.sort_values('name',ascending=True)

,trade,name,qty,price,active,reason,market,xdate
36,S,AIMIRT,10000,13.10,0,DOS,SET,2022-03-14
28,S,BCH,5000,22.40,0,5pct,SET100,2022-05-10
34,S,CPNCG,20000,12.50,0,DOS,SET,2022-02-14
26,S,HREIT,40100,9.00,0,DOS,SET,2022-03-04
25,S,JASIF,10000,11.30,0,DOS,SET,2022-03-03
31,S,KCE,1000,68.00,0,CUT,SET50,2022-03-21
29,S,PTT,3000,41.25,0,5pct,SET50,2022-03-03
27,S,RATCH,6000,46.00,0,DOS,SET50,2022-03-17
32,S,RCL,4000,47.50,0,CP1S,SET100,2022-03-23
30,S,SINGER,1500,52.75,0,25pct,SET100,2022-04-29


In [22]:
frames = [df_sts_b, df_sts_o, df_sts_s]
df_orders = pd.concat(frames, sort=False)
df_orders.sort_values(['trade','name'],ascending=True)

,trade,name,qty,price,active,reason,market,xdate
6,B,DOHOME,3000,19.70,0,RD15pct,SET100,2022-02-02
10,B,MAKRO,1500,39.00,0,RD05pct,SET,2022-03-03
17,B,STA,2500,26.25,0,RD20pct,SET100,2022-04-19
20,B,SYNEX,3000,23.80,0,RD15pct,SET100,2022-03-10
21,B,TISCO,500,97.75,0,ROUND,SET50,2022-04-29
22,B,TMT,15000,11.00,0,ROUND,SET,2022-04-18
0,O,AMATA,9000,20.60,0,3L,SET100,2022-04-25
1,O,ASIAN,10000,15.00,0,6L,SET,2022-04-29
3,O,BAM,12000,19.80,0,3L,SET100,2022-04-29
4,O,BBL,2400,130.00,0,1L,SET50,2022-04-22


In [23]:
df_orders.loc[df_orders.trade == 'O','trade'] = 'B'
df_orders.groupby(['trade']).count()

,name,qty,price,active,reason,market,xdate
trade,,,,,,,
B,14,14,14,14,14,14,14
S,12,12,12,12,12,12,12


In [24]:
file_name = 'orders.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(output_file, header=True, index=False)
df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(data_file, header=True, index=False)
df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(box_file, header=True, index=False)
df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(one_file, header=True, index=False)

In [25]:
rcds = df_orders.values.tolist()
len(rcds)

26

In [26]:
for rcd in rcds:
    print(rcd)

['B', 'DOHOME', 3000, 19.7, 0, 'RD15pct', 'SET100', '2022-02-02']
['B', 'MAKRO', 1500, 39.0, 0, 'RD05pct', 'SET', '2022-03-03']
['B', 'STA', 2500, 26.25, 0, 'RD20pct', 'SET100', '2022-04-19']
['B', 'SYNEX', 3000, 23.8, 0, 'RD15pct', 'SET100', '2022-03-10']
['B', 'TISCO', 500, 97.75, 0, 'ROUND', 'SET50', '2022-04-29']
['B', 'TMT', 15000, 11.0, 0, 'ROUND', 'SET', '2022-04-18']
['B', 'AMATA', 9000, 20.6, 0, '3L', 'SET100', '2022-04-25']
['B', 'ASIAN', 10000, 15.0, 0, '6L', 'SET', '2022-04-29']
['B', 'BAM', 12000, 19.8, 0, '3L', 'SET100', '2022-04-29']
['B', 'BBL', 2400, 130.0, 0, '1L', 'SET50', '2022-04-22']
['B', 'KKP', 3000, 66.0, 0, '3L', 'SET100', '2022-04-28']
['B', 'ORI', 18000, 10.8, 0, '3L', 'SET100', '2022-05-09']
['B', 'SPALI', 3000, 20.4, 0, 'RD05pct', 'SET100', '2022-04-27']
['B', 'SVI', 15000, 8.0, 0, '6L', 'SET', '2022-05-05']
['S', 'JASIF', 10000, 11.3, 0, 'DOS', 'SET', '2022-03-03']
['S', 'HREIT', 40100, 9.0, 0, 'DOS', 'SET', '2022-03-04']
['S', 'RATCH', 6000, 46.0, 0, 'DO

In [27]:
sql = """
INSERT INTO orders (trade, name, qty, price, active,
reason, market, xdate)
VALUES (?, ?, ?, ?, ?, ?, ?, ?)"""
print(sql)


INSERT INTO orders (trade, name, qty, price, active,
reason, market, xdate)
VALUES (?, ?, ?, ?, ?, ?, ?, ?)


In [28]:
for rcd in rcds:
    conlite.execute(sql, rcd)

### After call Ord-log.bat

In [29]:
def categorise(row):  
    if row['trade'] == 'B':
        return 'Buy'
    else:
        return 'Sell'

In [30]:
df = pd.read_csv('../data/orders-log.csv')
df['trans'] = df.apply(lambda row: categorise(row), axis=1)

df_tab = pn.widgets.Tabulator(df[colt], layout='fit_data', width=800)
df_tab

Tabulator(layout='fit_data', value=   trans    name  ..., width=800)

In [31]:
df[colt]

,trans,name,spd,reason,qty,target,current,percent,active,xdate
0,Buy,AMATA,-14,3L,10000,20.00,21.40,0.00%,0,2022-04-25
1,Buy,ASIAN,-4,6L,10000,15.00,15.40,0.00%,0,2022-04-29
2,Buy,BAM,1,NaN,10000,20.50,20.40,-2.86%,0,2022-04-29
3,Buy,BBL,-12,1L,2400,130.00,136.00,-0.73%,0,2022-04-22
4,Buy,DOHOME,-16,RD15pct,3000,19.70,21.30,0.00%,0,2022-02-02
5,Buy,KKP,-12,3L,3000,65.00,68.00,-0.37%,0,2022-04-28
6,Buy,MAKRO,-2,RD05pct,1500,39.00,39.50,0.00%,0,2022-03-03
7,Buy,ORI,-12,3L,20000,10.00,11.20,0.00%,0,2022-05-09
8,Buy,SPALI,-16,RD05pct,3000,20.40,22.00,-0.45%,0,2022-04-27
9,Buy,STA,-2,RD20pct,2500,26.25,26.75,0.00%,0,2022-04-19
